In [16]:
from __future__ import annotations
from dataclasses import dataclass, field

from typing import Optional, List

@dataclass
class File:
    name : str
    size : int


@dataclass
class Directory:
    path: str
    parent_directory: Directory = None 
    subdirectories : List[Directory] = field(default_factory=list)
    files : List[File] = field(default_factory=list)

    def size(self):
        return sum((f.size for f in self.files)) + sum(d.size() for d in self.subdirectories)




In [52]:
from lib.utils import iter_file


class Interpreter:
    def __init__(self) -> None:
        self.current_directory : Optional[Directory] = None
        self.root = None
    
    def parse(self, l : str) -> None:
        if l.startswith('$'):
            self.parse_command(l[2:])
        else:
            self.parse_output(l)
        
    def parse_cd(self, l : str) -> None:
        path = l[3:]
        if path == '..':
            if self.current_directory.parent_directory is not None:
                self.current_directory = self.current_directory.parent_directory
        elif path.startswith('/'):
            self.current_directory = Directory(path)
            if path == '/':
                self.root = self.current_directory
        else:
            d = Directory(f'{self.current_directory.path}/{path}', parent_directory=self.current_directory)
            self.current_directory.subdirectories.append(d)
            self.current_directory = d

    def parse_command(self, l : str) -> None:
        if l.startswith('cd'):
            self.parse_cd(l)
        elif l.startswith('ls'):
            ...

    def parse_output(self, l : str) -> None:
        if l.startswith('dir'):
            return
        file_size, file_name = l.split(" ")
        self.current_directory.files.append(File(file_name, int(file_size)))


interpreter = Interpreter()
for l in iter_file('../day7.txt'):
    interpreter.parse(l)

big_dirs = []

def find_big_dirs(root : Directory, big_dirs : list[Directory]):
    for d in root.subdirectories:
        if d.size() <= 100000:
            big_dirs.append(d)
        find_big_dirs(d, big_dirs)

find_big_dirs(interpreter.root, big_dirs)

/lnj
/nhb
/ccsfm
/jlpbbds
/nzcnb
/snq
/tmjnvcbl
/gclzbvt
/jtfddbs
/lsss
/cqmbtj
/nsq
/fwfcmfbz
/jqz
/nrmp
/gbhcnts
/qdsrs
/frlj
/ljvwzj
/stt
/tfpl
/wpfj
/gbhcnts
/cthptwcf
/qtbprrq
/tvhz
/ndgqtvhg
/sljbrmhb
/stp
/ppqjzln


In [53]:
sum((d.size() for d in big_dirs))

1490523

In [54]:
TOTAL_SIZE = 70000000
MIN_SIZE = 30000000

In [55]:
remaining = TOTAL_SIZE - interpreter.root.size()

In [56]:
min_folder_size_to_delete = MIN_SIZE - remaining

In [57]:
min_folder_size_to_delete

9192532

In [58]:

def find_min_folder_to_delete(root : Directory) -> Directory:
        
    directory_to_delete = root  if root.size() >= min_folder_size_to_delete else None
    for d in root.subdirectories:
        curr_dir = find_min_folder_to_delete(d)
        if not directory_to_delete or curr_dir and curr_dir.size() < directory_to_delete.size():
            directory_to_delete = curr_dir
    return directory_to_delete 
            



In [60]:

find_min_folder_to_delete(interpreter.root).size()

12390492

In [39]:
current_deletion